In [ ]:
# --- SEL 1: HUBUNGKAN DRIVE & INSTAL DEPENDENSI KRUSIAL ---
# Perintah ini akan memastikan dependensi inti terinstal dengan benar.
# Jalankan Sel 1 ini terlebih dahulu setelah restart baru jalankan semua sel secara berurutan

# Hubungkan Google Drive.
from google.colab import drive
drive.mount('/content/drive')

# Hapus semua instalasi terkait untuk menghindari konflik.
# Hapus juga numpy agar kita bisa menginstal versi spesifik.
!pip uninstall -y basicsr facexlib gfpgan realesrgan torch torchvision numpy
!pip cache purge

# Instal NumPy versi 1.26.4 terlebih dahulu dan secara paksa.
# Penting: Instal juga torch dan torchvision DI SINI dengan versi yang kompatibel.
print("\nInstalling numpy==1.26.4, torch==2.2.2, and torchvision==0.17.2 with force-reinstall...")
!pip install numpy==1.26.4 --force-reinstall

print("\n--- INSTALASI INTI SELESAI ---")
print("SILAKAN RESTART RUNTIME SEKARANG (Runtime > Restart session).")
print("Setelah restart, jalankan Sel 2.")

In [ ]:
# --- SEL 2: INSTAL SISA DEPENDENSI DAN MODEL (DENGAN --no-deps untuk semuanya) ---
# Jalankan sel ini SETELAH RESTART SESI DARI SEL 1.

!pip install torch==2.2.2 torchvision==0.17.2 --index-url https://download.pytorch.org/whl/cu121


# Instal basicsr dari repositori GitHub.
print("\nInstalling basicsr from its GitHub repository (without dependencies)...")
!pip install git+https://github.com/xinntao/BasicSR.git --no-deps

# Instal dependensi lainnya dengan --no-deps untuk mencegah konflik NumPy.
# Ini mungkin menyebabkan error jika ada dependensi lain yang belum terinstal.
print("\nInstalling facexlib, gfpgan, realesrgan (with --no-deps)...")
!pip install facexlib --no-deps
!pip install gfpgan --no-deps
!pip install realesrgan --no-deps

# Unduh model pra-terlatih.
print("\nDownloading the pre-trained models...")
!mkdir -p ~/.cache/torch/hub/checkpoints/
!wget -c https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -P ~/.cache/torch/hub/checkpoints/

print("\nSetup complete! All dependencies are in place.")

In [ ]:
# --- SEL 3: KONFIGURASI DAN JALANKAN UPSCALING (4x) ---

import os
import glob
import numpy as np
import torch
import shutil
from realesrgan import RealESRGANer
from basicsr.archs.rrdbnet_arch import RRDBNet
from PIL import Image

# --- PENTING: UBAH PATH FOLDER INI! ---
# Ganti 'INPUT_FOLDER', 'OUTPUT_FOLDER', dan 'PROCESSED_FOLDER' dengan nama folder Anda.
base_folder_path = '/content/drive/MyDrive'
# ----------------------------------------------------

# Nama sub-folder (bisa diubah jika perlu)
input_folder_name = "source"
output_folder_name = "result"
processed_folder_name = "done"

# Membuat path lengkap ke setiap sub-folder secara otomatis
input_folder_path = os.path.join(base_folder_path, input_folder_name)
output_folder_path = os.path.join(base_folder_path, output_folder_name)
processed_folder_path = os.path.join(base_folder_path, processed_folder_name)


# Buat folder output dan processed jika belum ada.
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)
    print(f"Created output directory: {output_folder_path}")

if not os.path.exists(processed_folder_path):
    os.makedirs(processed_folder_path)
    print(f"Created processed directory: {processed_folder_path}")

# --- Initialize Real-ESRGANer ---
print("\nInitializing Real-ESRGAN upsampler for 4x scale...")
model_name = 'RealESRGAN_x4plus'
model_path = os.path.expanduser(f"~/.cache/torch/hub/checkpoints/{model_name}.pth")

if not os.path.exists(model_path):
    raise FileNotFoundError(f"Model file not found at {model_path}. Please check the download step.")

print(f"Loading model from: {model_path}")
model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=23, num_grow_ch=32, scale=4)
upsampler = RealESRGANer(
    scale=4,
    model_path=model_path,
    model=model,
    tile=512,
    tile_pad=10,
    pre_pad=0,
    half=True,
    device=torch.device('cuda')
)

# --- Proses semua gambar di folder input ---
print("\nStarting the upscaling process...")
image_extensions = ['jpg', 'jpeg', 'png']
image_paths = []
for ext in image_extensions:
    image_paths.extend(glob.glob(f'{input_folder_path}/*.{ext}'))
    image_paths.extend(glob.glob(f'{input_folder_path}/*.{ext.upper()}'))

if not image_paths:
    print("No images found in the input folder. Please check your folder path and image extensions.")
else:
    # --- Tambahkan ini: Atur kualitas JPG yang diinginkan (misal: 90) ---
    jpg_quality = 95 # Kamu bisa mengubah nilai ini antara 1 hingga 95
    # ------------------------------------------------------------------

    for img_path in image_paths:
        try:
            print(f"Processing {os.path.basename(img_path)}...")
            img_pil = Image.open(img_path).convert('RGB')
            img_np = np.array(img_pil)

            output_img, _ = upsampler.enhance(img_np, outscale=4)

            # Mendapatkan format asli file
            base_name = os.path.splitext(os.path.basename(img_path))[0]
            original_ext = os.path.splitext(img_path)[1].lower() # .lower() untuk memastikan konsistensi

            # Simpan hasil upscale dengan format asli
            output_path = os.path.join(output_folder_path, f"{base_name}{original_ext}")

            # --- Bagian yang dimodifikasi untuk mengatur kualitas JPG ---
            if original_ext in ['.jpg', '.jpeg']:
                Image.fromarray(output_img).save(output_path, quality=jpg_quality, subsampling=0) # subsampling=0 untuk kualitas terbaik
            else:
                Image.fromarray(output_img).save(output_path)
            # ----------------------------------------------------------

            # Pindahkan file asli ke folder 'processed'
            shutil.move(img_path, processed_folder_path)

            print(f"-> Saved upscale to {output_path} and moved original to {processed_folder_path}")

        except Exception as e:
            print(f"Error processing {os.path.basename(img_path)}: {e}")

print("\nUpscaling process finished!")
print(f"Results are saved in: {output_folder_path}")